<a href="https://colab.research.google.com/github/filmerxyz/JSTP-22_SkinDiseaseClassificationUsingMachineLearning/blob/master/USE_this_Almost_Accumulator_DenseNet201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Check GPU

In [1]:
!nvidia-smi

Sun May 24 19:16:37 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.82       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
!pip install runai

# Import Libraries

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
import pandas as pd
import numpy as np
import os
import math
import itertools
from PIL import Image
from matplotlib import pyplot as plt

import keras
from keras import backend as K
from keras.optimizers import Adam, Optimizer
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import DenseNet201
from keras.legacy import interfaces

from sklearn.model_selection import train_test_split
from sklearn.utils import compute_sample_weight, compute_class_weight
from sklearn.metrics import balanced_accuracy_score, classification_report, confusion_matrix

Using TensorFlow backend.


In [5]:
keras.__version__

'2.3.1'

In [6]:
!git clone https://github.com/EvilPickle-PCSHSPT/ham10000-with-one-image-folder

fatal: destination path 'ham10000-with-one-image-folder' already exists and is not an empty directory.


In [0]:
metadata_path = '/content/ham10000-with-one-image-folder/HAM10000_metadata.csv'
img_path = '/content/ham10000-with-one-image-folder/HAM1000_images'

In [0]:
label_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']

In [0]:
data = pd.read_csv(metadata_path)

In [0]:
num_examples = data.values.shape[0]

In [0]:
inputs = np.empty(shape=(num_examples, 224, 224, 3), dtype=np.float32)
labels = np.empty(shape=(num_examples), dtype=np.uint8)

### Resize to 224*224

In [0]:
for i, row in enumerate(data.values):
    img_id = row[1]
    label = row[2]
    all_img_path = os.path.join(img_path, img_id) + '.jpg' 
    if (os.path.isfile(all_img_path)):
        im_path = all_img_path
    else:
        raise Exception ('File not found \'%s\'' %img_id)
    img = Image.open(im_path).resize((224, 224), Image.LANCZOS)
    inputs[i] = np.array(img)/255.0
    labels[i] = label_names.index(label)

### Split Data

In [0]:
x_train, x_test, y_train, y_test = train_test_split(inputs, labels, test_size=0.1, random_state=2019)

del inputs
del labels

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3, random_state=2019)

In [14]:
print(x_train.shape, y_train.shape)
print(x_val.shape, y_val.shape)
print(x_test.shape, y_test.shape)

(6309, 224, 224, 3) (6309,)
(2704, 224, 224, 3) (2704,)
(1002, 224, 224, 3) (1002,)


# Hyperparameters

In [0]:
BATCH_SIZE = 16
LR = 1e-3
EPOCHS = 100

# Data Augmentation

In [0]:
datagen = ImageDataGenerator(
        rotation_range=360,
        zoom_range=0.3,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='reflect')

In [0]:
datagen.fit(x_train)

# Adam Accumulate

In [0]:
import keras.optimizers
import runai.ga.keras

In [19]:
optimizer = runai.ga.keras.optimizers.Adam(steps=16)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


[Run:AI] [DEBUG   ] [24-05-2020 19:19:24.075] [323] [optimizers.py          :16  ] Wrapping 'Adam' Keras optimizer with GA of 16 steps


# Build Model

In [20]:
base_model = DenseNet201(include_top=False, weights='imagenet', input_shape=(224,224,3))

# base_model.trainable = False

# x = base_model.output
# x = GlobalAveragePooling2D()(x)
# preds = Dense(7,activation='softmax')(x)

# model = Model(inputs=base_model.input,outputs=preds)

# model.compile(optimizer=optimizer, 
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])

In [0]:
base_model.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)
preds = Dense(7,activation='softmax')(x)

model = Model(inputs=base_model.input,outputs=preds)

model.compile(optimizer=optimizer, 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
# model.summary()

### Callbacks

In [0]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, verbose=0, mode='min')

model_checkpoint = keras.callbacks.ModelCheckpoint(filepath='acc_densenet201_model.{epoch:02d}-{val_loss:.2f}.h5', save_best_only=True, monitor='val_loss', mode='min')

reduce_lr_rate = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, min_lr=0.00001 , patience=10, verbose=1, min_delta=1e-4, mode='min')

### Training

In [26]:
history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=BATCH_SIZE),
                              validation_data=(x_val, y_val, compute_sample_weight('balanced', y_val)),
                              class_weight=compute_class_weight('balanced', np.unique(y_train), y_train),
                              steps_per_epoch=math.ceil(x_train.shape[0]/BATCH_SIZE),
                              epochs=EPOCHS,
                              callbacks=[early_stop, model_checkpoint, reduce_lr_rate])

Epoch 1/100
395/395 [==============================] - 149s 376ms/step - loss: 0.6366 - accuracy: 0.7657 - val_loss: 1.2051 - val_accuracy: 0.7888
Epoch 2/100
395/395 [==============================] - 148s 374ms/step - loss: 0.5784 - accuracy: 0.7873 - val_loss: 1.4833 - val_accuracy: 0.7833
Epoch 3/100
395/395 [==============================] - 148s 375ms/step - loss: 0.5242 - accuracy: 0.8033 - val_loss: 1.0723 - val_accuracy: 0.7837
Epoch 4/100
 38/395 [=>............................] - ETA: 2:02 - loss: 0.5065 - accuracy: 0.8109

KeyboardInterrupt: ignored